### Descrição - Modelo Time Dependent (ETS/ARIMA/SARIMA)

Separa todo o conjunto de dados fornecidos ``train.csv`` para divider em treinamento e teste. Utiliza modelos **ETS** e **ARIMA** para regressão temporal e forecasting


### Libraries

In [ ]:
from warnings import simplefilter
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
from random import sample
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from xgboost import XGBRegressor

simplefilter("ignore")

In [ ]:
# Set Matplotlib defaults
plt.style.use("seaborn-whitegrid")
plt.rc(
    "figure",
    autolayout=True,
    figsize=(10, 6),
    titlesize=18,
    titleweight='bold'
    )
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=16,
    titlepad=10,
    )
plot_params = dict(
    color="0.75",
    style=".-",
    markeredgecolor="0.25",
    markerfacecolor="0.25",
    )


### Import dataset

In [ ]:
cols = ["cfips", "microbusiness_density", "active"]
train_df = pd.read_csv('./files/train.csv',
                      usecols=["first_day_of_month"] + cols,
                      parse_dates=['first_day_of_month'],
                      index_col='first_day_of_month',
                      ).to_period('D').reindex(columns=cols)

train_df.index.names = ['Month']
# retail = pd.concat({'Sales': retail}, names=[None, 'Industries'], axis=1)
train_df = train_df.drop("active", axis=1)

print(train_df.info())
train_df.head()

### Trend model

#### Pivoted dataframe

In [ ]:
# make pivot, setting date by index and cfips in columns
train_pivoted = train_df.pivot_table(index='Month', columns='cfips', values='microbusiness_density')
train_pivoted.head()

#### Train/Test split

In [ ]:
# divide em treino e teste
X_train = train_pivoted.copy()
X_test = train_pivoted.iloc[-4:, :]
X_train.index = X_train.index.to_timestamp()
# X_train.info()

#### SMAPE

In [ ]:
# calculate SMAPE between forecasts and actual values
def smape_error(y_pred, y_true):
    numerator = 2 * np.abs(y_true.values - y_pred.values)
    denominator = np.abs(y_pred.values) + np.abs(y_true.values)
    
    df = pd.DataFrame(index=range(max(numerator.shape)), columns=["Numerator", "Denominator"])
    df["Numerator"] = numerator.T
    df["Denominator"] = denominator.T
    
    df = df[df["Denominator"] != 0.0]   # Drop rows where column "Denominator" is equal to 0

    return 100 * np.mean(df["Numerator"].values/df["Denominator"].values)


### ETS Model

In [ ]:
def best_ets_params(df_train, df_test):
    
    trend = ['add', 'mul', None]
    damped_trend = [False, True]
    seasonal = ['add', 'mul', None]
    #seasonal_periods = [12]
    #initialization_method = ['estimated'] # ['estimated', 'heuristic']

    # return all possible combinations between the parameters
    params_comb = list(itertools.product(trend, damped_trend, seasonal, seasonal_periods, initialization_method))[:-3]
    
    df = pd.DataFrame(index=list(df_train.columns),
                      columns=['smape_ets_train', 'min_smape_ets_test', 'trend', 'damped_trend',
                               'seasonal', 'seasonal_periods', 'init_method']
                     )
    
    df.index.name = df_train.columns.name
    i = 0
    
    for cfip in list(df.index)[2000:2500]:
        error_train, error_test = [], []
        
        for params in params_comb:
            try:
                model = ExponentialSmoothing(df_train[cfip], trend=params[0], damped_trend=params[1], seasonal=params[2],
                                             seasonal_periods=params[3], initialization_method=params[4],
                                            ).fit(optimized=True,
                                                  use_brute=True,
                                                  remove_bias=True,
                                                  method='bh',
                                                 )
                
                fitted_values = model.fittedvalues
                forecasts = model.forecast(4)
                error_train.append(smape_error(fitted_values, df_train[cfip]))
                error_test.append(smape_error(forecasts, df_test[cfip]))
                
            except:
                model = ExponentialSmoothing(df_train[cfip], trend=params[0], damped_trend=params[1], seasonal=params[2],
                                             seasonal_periods=params[3], initialization_method=params[4],
                                            ).fit()
                
                fitted_values = model.fittedvalues
                forecasts = model.forecast(4)
                error_train.append(smape_error(fitted_values, df_train[cfip]))
                error_test.append(smape_error(forecasts, df_test[cfip]))
        
        i+=1
        print(i, len(error_train), len(error_test), len(params_comb))
        # df.loc[cfip, :] = [error_train[0], error_test[0]] + list(params_comb[0])
        idx = error_test.index(min(error_test))
        df.loc[cfip, :] = [error_train[idx]] + [error_test[idx]] + list(params_comb[idx])
    
    return df

df = best_ets_params(X_train, X_test)
df.to_csv('./files/ETS_best_params_CV.csv', header=True, sep=",", index=True)


In [ ]:
df.dropna().mean()

In [ ]:
ets_df = pd.read_csv('./files/ETS_initial_params.csv', index_col=['cfips'], dtype={'seasonal_periods': np.int64})
ets_df.iloc[:600].mean()
# U = ets_df.sort_values('smape_ets_test', ascending=False)
# U

In [ ]:
print(U['smape_ets_test'].mean())

plt.figure(figsize=(8,6))
axs = X_train[17093].plot(label='Original data', color='C0', style='.-', sharex=True)
X_test[17093].plot(label='X_test', color='C0', ax=axs)
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
axs = X_train[42101].plot(label='Original data', color='C0', style='.-', sharex=True)
X_test[42101].plot(label='X_test', color='C0', ax=axs)
forecasts.plot(label='ETS Model', style='.-', color='C1', ax=axs)
model.fittedvalues.plot(label='fitted values', style='.-', color='C3')
plt.title('cfip 42101 - Exponencial Smoothing' , fontsize=16)
# plt.ylabel("Monthly Sales Amount");
plt.xlabel("Month")
plt.legend();

In [ ]:
residuals = X_train[42101] # - model.fittedvalues

res_df = pd.DataFrame(residuals) #columns=['residual']
res_df['lag1'] = res_df[42101].shift(1)
res_df.dropna(inplace=True)
res_df.head()

In [ ]:
res_df.plot(kind='scatter', x='lag1', y=42101);

### ARIMA model

### Augmented Dickey Fuller Test

Requirements for Stationary Series:

- **Critical Value (5%)  >  Test Statistic**
- **p-value  <  0.05**


In [ ]:
def adf_reqs(train_df):
    
    df = pd.DataFrame(index=train_df.columns,
                      columns=['test_statistic','p_value', 'critical_value_5pct'])
    
    for i in df.index:
        
        adftest = adfuller(train_df[i], autolag='AIC')
        df.loc[i,:] = [adftest[0], adftest[1], adftest[4]['5%']]        
    
    for col in df.columns: df[col] = df[col].astype(float)
    
    df_notstat = df.loc[(df.p_value > .05) & (df.test_statistic > df.critical_value_5pct)]
    df_notstat['stationary'] = 'not'
    df_notstat.drop(['test_statistic','p_value', 'critical_value_5pct'], axis=1, inplace=True)
    
    df = df.merge(df_notstat, how='left', on='cfips').fillna('yes')
    
    return df


In [ ]:
adf_df = adf_reqs(train_pivoted)

In [ ]:
decomposition = sm.tsa.seasonal_decompose(X_train[42101], model='additive')
fig = decomposition.plot()
plt.show()

In [ ]:
adfuller(X_train[cfip], autolag='AIC')

In [ ]:
adfuller(X_train[cfip].diff().dropna(), autolag='AIC')

In [ ]:
plot_acf(X_train[cfip].diff().dropna(), lags=12);

In [ ]:
plot_pacf(X_train[cfip].diff().dropna(), lags=12);

In [ ]:
cfip=1007
model = ARIMA(X_train[cfip], order=(2,0,2)).fit()
model.summary()

In [ ]:
pred = model.predict()
residuals = X_train[cfip] - pred
fore = model.predict(start=len(X_train[cfip]), end=(len(train_pivoted[cfip])-1+4))
print(fore)


In [ ]:
X_train[cfip].plot(**plot_params)
X_test[cfip].plot(**plot_params)
pred[1:].plot()
fore.plot(color='C1')
plt.show()


In [ ]:
residuals[1:].plot()
plt.show()

In [ ]:
res_df = pd.DataFrame(residuals[1:], columns=['residual'])
res_df['lag1'] = res_df.residual.shift(1)
res_df.dropna(inplace=True)
res_df.head()

In [ ]:
res_df.plot(kind='scatter', x='lag1', y='residual')

### Iterate (p, d, q) ARIMA model

In [ ]:
def best_pdq(df_train, df_pivoted, df_test):
    
    p, d, q = range(0,3), range(0,2), range(0,3)
    pdq_comb = list(itertools.product(p,d,q))[1:]   # return all possible combinations between the parameters
    pdq_comb
    
    df = pd.DataFrame(index=list(df_train.columns), columns=['trainset_smape', 'testset_min_smape', 'p', 'd', 'q'])
    df.index.name = df_train.columns.name
    i = 0
    for cfip in list(df.index):
        error_train, error_test = [], []
        
        for pdq in pdq_comb:
            try:
                model = ARIMA(df_train[cfip], order=pdq).fit()
                fitted_values = model.predict()
                forecasts = model.predict(start=len(df_train[cfip]), end=(len(df_pivoted[cfip])-1))   # 2022-07 to 2022-10
                error_train.append(smape_error(fitted_values, df_train[cfip]))
                error_test.append(smape_error(forecasts, df_test[cfip]))
                
            except:
                error_train.append(201)
                error_test.append(201)
                
        i+=1
        print(i, len(error_train), len(error_test), len(pdq_comb))
        
        idx = error_test.index(min(error_test))
        df.loc[cfip, :] = [error_train[idx]] + [error_test[idx]] + list(pdq_comb[idx])
        
    return df

df = best_pdq(X_train, train_pivoted, X_test)
df.to_csv('./files/ARIMA_best_pdq_p0-2_d0-1_q0-2_all-cfips.csv', header=True, sep=",", index=True)


### SARIMA Model

In [ ]:
model_sarima = SARIMAX(X_train[42101], order=([9],1,0), seasonal_order=(0, 0, 1, 12)).fit()
model_sarima.summary()

In [ ]:
pred = model_sarima.predict()
residuals = X_train[42101] - pred
fore = model_sarima.predict(start=len(X_train[42101]), end=(len(train_pivoted[42101])-1))
print(fore)

In [ ]:
X_train[42101].plot(**plot_params)
X_test[42101].plot(**plot_params)
pred[1:].plot()
fore.plot(color='C1')
plt.show()

In [ ]:
df1 = pd.read_csv('./files/ARIMA_best_pqd_p1-7_q1-7_d1-7_range_pt0_600.csv', index_col='cfips').dropna()
for i in df1.columns[:-1]: df1[i] = df1[i].astype(np.int32)

df2 = pd.read_csv('./files/ARIMA_best_pqd_p1-7_q1-7_d1-7_range_pt600_1200.csv', index_col='cfips').dropna()
for i in df2.columns[:-1]: df2[i] = df2[i].astype(np.int32)

df3 = pd.read_csv('./files/ARIMA_best_pqd_p1-7_q1-7_d1-7_range_pt1200_end.csv', index_col='cfips').dropna()
for i in df3.columns[:-1]: df3[i] = df3[i].astype(np.int32)

df_all = pd.concat([df1, df2, df3], axis=0)
df_all = df_all.rename(columns={'q': 'd', 'd': 'q'})
df_all.to_csv('./files/ARIMA_best_pdq_p1-7_d1-7_q1-7_all-cfips.csv', header=True, sep=",", index=True)
